# 迁移学习实战：花卉分类器

迁移学习是深度学习中最实用的技术之一，它允许我们：
1. **利用预训练知识** - 使用在大数据集上学到的特征
2. **减少训练数据需求** - 小数据集也能训练好模型
3. **加速收敛** - 训练更快，效果更好

本教程使用TensorFlow Flowers数据集，演示完整的迁移学习流程：
- 加载预训练模型（MobileNetV2）
- 冻结基础层并添加自定义分类头
- 两阶段训练策略
- 微调(Fine-tuning)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import pathlib

# 设置随机种子
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

print(f"TensorFlow版本: {tf.__version__}")
print(f"GPU可用: {tf.config.list_physical_devices('GPU')}")

## 第一部分：数据准备

### 1.1 下载TensorFlow Flowers数据集

数据集包含5类花卉图像：
- 雏菊(daisy)
- 蒲公英(dandelion)
- 玫瑰(roses)
- 向日葵(sunflowers)
- 郁金香(tulips)

In [ ]:
# 下载花卉数据集
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = keras.utils.get_file(
    'flower_photos',
    origin=dataset_url,
    untar=True
)
data_dir = pathlib.Path(data_dir)

# 统计图像数量
image_count = len(list(data_dir.glob('*/*.jpg')))
print(f"数据集路径: {data_dir}")
print(f"总图像数量: {image_count}")

In [ ]:
# 显示每个类别的图像数量
class_names = sorted([item.name for item in data_dir.glob('*') if item.is_dir()])
print(f"\n类别数量: {len(class_names)}")
print(f"类别名称: {class_names}")

print("\n各类别图像数量:")
for class_name in class_names:
    count = len(list((data_dir / class_name).glob('*.jpg')))
    print(f"  {class_name}: {count}张")

### 1.2 创建数据集

In [ ]:
# 超参数设置
BATCH_SIZE = 32
IMG_SIZE = (160, 160)  # MobileNetV2推荐输入尺寸
VALIDATION_SPLIT = 0.2

# 创建训练数据集
train_ds = keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=VALIDATION_SPLIT,
    subset='training',
    seed=RANDOM_SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

# 创建验证数据集
val_ds = keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=VALIDATION_SPLIT,
    subset='validation',
    seed=RANDOM_SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
num_classes = len(class_names)
print(f"类别: {class_names}")

In [ ]:
# 可视化部分训练图像
plt.figure(figsize=(12, 8))
for images, labels in train_ds.take(1):
    for i in range(12):
        ax = plt.subplot(3, 4, i + 1)
        plt.imshow(images[i].numpy().astype('uint8'))
        plt.title(class_names[labels[i]])
        plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# 数据集性能优化
# - cache: 将数据缓存到内存
# - prefetch: 在训练时预取下一批数据
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## 第二部分：构建迁移学习模型

### 2.1 数据增强层

数据增强可以增加训练数据的多样性，减少过拟合

In [ ]:
# 创建数据增强层
data_augmentation = keras.Sequential([
    keras.layers.RandomFlip('horizontal'),
    keras.layers.RandomRotation(0.2),
    keras.layers.RandomZoom(0.2),
], name='data_augmentation')

# 可视化数据增强效果
plt.figure(figsize=(12, 8))
for images, _ in train_ds.take(1):
    first_image = images[0]
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
        plt.imshow(augmented_image[0].numpy().astype('uint8'))
        plt.axis('off')
plt.suptitle('数据增强效果示例')
plt.tight_layout()
plt.show()

### 2.2 加载预训练模型

使用MobileNetV2作为基础模型：
- 轻量级，适合快速训练
- 在ImageNet上预训练
- `include_top=False`表示不包含最后的分类层

In [ ]:
# 预处理函数
# MobileNetV2期望输入在[-1, 1]范围
preprocess_input = keras.applications.mobilenet_v2.preprocess_input

# 加载预训练的MobileNetV2
IMG_SHAPE = IMG_SIZE + (3,)

base_model = keras.applications.MobileNetV2(
    input_shape=IMG_SHAPE,
    include_top=False,      # 不包含分类层
    weights='imagenet'      # 使用ImageNet预训练权重
)

# 查看基础模型输出
print(f"基础模型输入形状: {base_model.input_shape}")
print(f"基础模型输出形状: {base_model.output_shape}")
print(f"基础模型总层数: {len(base_model.layers)}")
print(f"基础模型参数量: {base_model.count_params():,}")

In [ ]:
# 查看基础模型对输入图像的特征提取
for images, _ in train_ds.take(1):
    image_batch = images[:1]
    processed = preprocess_input(image_batch)
    feature_batch = base_model(processed)
    print(f"输入图像形状: {image_batch.shape}")
    print(f"特征图形状: {feature_batch.shape}")
    print(f"每张图像产生 {feature_batch.shape[1]}x{feature_batch.shape[2]}x{feature_batch.shape[3]} 的特征图")

### 2.3 冻结基础模型

**冻结权重的原因**：
预训练模型的权重是精心调整过的。新添加的分类层是随机初始化的。
如果不冻结，随机初始化层产生的大梯度会破坏预训练权重。

In [ ]:
# 冻结基础模型的所有层
base_model.trainable = False

# 验证冻结状态
print(f"可训练变量数量: {len(base_model.trainable_variables)}")
print(f"不可训练变量数量: {len(base_model.non_trainable_variables)}")

### 2.4 添加自定义分类头

In [ ]:
def build_transfer_model(base_model, num_classes, dropout_rate=0.2):
    """
    构建迁移学习模型
    
    Parameters
    ----------
    base_model : keras.Model
        预训练的基础模型
    num_classes : int
        分类类别数
    dropout_rate : float
        Dropout比率
    
    Returns
    -------
    keras.Model
        完整的迁移学习模型
    """
    inputs = keras.Input(shape=IMG_SHAPE)
    
    # 数据增强（仅训练时）
    x = data_augmentation(inputs)
    
    # 预处理
    x = preprocess_input(x)
    
    # 基础模型（特征提取）
    # training=False确保BN层使用预训练的统计量
    x = base_model(x, training=False)
    
    # 分类头
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dropout(dropout_rate)(x)
    outputs = keras.layers.Dense(num_classes, activation='softmax')(x)
    
    model = keras.Model(inputs, outputs)
    return model

# 构建模型
model = build_transfer_model(base_model, num_classes)
model.summary()

## 第三部分：训练模型

### 3.1 第一阶段：只训练分类头

In [ ]:
# 编译模型
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 第一阶段训练
INITIAL_EPOCHS = 5

print("第一阶段：训练分类头（基础模型冻结）")
print(f"可训练参数: {sum([np.prod(v.shape) for v in model.trainable_variables]):,}")

history = model.fit(
    train_ds,
    epochs=INITIAL_EPOCHS,
    validation_data=val_ds
)

In [ ]:
# 评估第一阶段结果
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

print(f"\n第一阶段结果:")
print(f"训练准确率: {acc[-1]:.4f}")
print(f"验证准确率: {val_acc[-1]:.4f}")

### 3.2 第二阶段：微调(Fine-tuning)

微调时解冻部分基础模型层，使用更低的学习率继续训练

In [ ]:
# 解冻基础模型
base_model.trainable = True

# 只微调最后几层，保持前面的层冻结
# MobileNetV2有155层，我们只解冻最后50层
fine_tune_at = 100

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

print(f"基础模型总层数: {len(base_model.layers)}")
print(f"冻结层数: {fine_tune_at}")
print(f"可训练层数: {len(base_model.layers) - fine_tune_at}")
print(f"总可训练参数: {sum([np.prod(v.shape) for v in model.trainable_variables]):,}")

In [ ]:
# 使用更低的学习率重新编译
# 微调时学习率应该比初始训练低10倍以上
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 第二阶段训练
FINE_TUNE_EPOCHS = 5
TOTAL_EPOCHS = INITIAL_EPOCHS + FINE_TUNE_EPOCHS

print("\n第二阶段：微调")
history_fine = model.fit(
    train_ds,
    epochs=TOTAL_EPOCHS,
    initial_epoch=INITIAL_EPOCHS,
    validation_data=val_ds
)

In [ ]:
# 合并训练历史
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']
loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

## 第四部分：评估与可视化

In [ ]:
# 绘制训练曲线
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 准确率曲线
axes[0].plot(acc, label='训练准确率')
axes[0].plot(val_acc, label='验证准确率')
axes[0].axvline(x=INITIAL_EPOCHS-1, color='gray', linestyle='--', label='微调开始')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].set_title('训练和验证准确率')
axes[0].legend()
axes[0].grid(True)

# 损失曲线
axes[1].plot(loss, label='训练损失')
axes[1].plot(val_loss, label='验证损失')
axes[1].axvline(x=INITIAL_EPOCHS-1, color='gray', linestyle='--', label='微调开始')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].set_title('训练和验证损失')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# 在验证集上评估
final_loss, final_accuracy = model.evaluate(val_ds, verbose=0)
print(f"最终验证损失: {final_loss:.4f}")
print(f"最终验证准确率: {final_accuracy:.4f}")

In [ ]:
# 可视化预测结果
plt.figure(figsize=(15, 10))

for images, labels in val_ds.take(1):
    predictions = model.predict(images, verbose=0)
    predicted_classes = np.argmax(predictions, axis=1)
    
    for i in range(12):
        ax = plt.subplot(3, 4, i + 1)
        plt.imshow(images[i].numpy().astype('uint8'))
        
        true_label = class_names[labels[i]]
        pred_label = class_names[predicted_classes[i]]
        confidence = predictions[i][predicted_classes[i]] * 100
        
        color = 'green' if labels[i] == predicted_classes[i] else 'red'
        plt.title(f'真实: {true_label}\n预测: {pred_label} ({confidence:.1f}%)', 
                  color=color, fontsize=9)
        plt.axis('off')

plt.tight_layout()
plt.show()

## 总结

### 迁移学习关键步骤

1. **加载预训练模型** - 使用`include_top=False`去除分类层
2. **冻结基础模型** - `base_model.trainable = False`
3. **添加自定义分类头** - GlobalAveragePooling + Dense
4. **第一阶段训练** - 只训练分类头，使用正常学习率
5. **微调** - 解冻部分层，使用更低学习率继续训练

### 实践建议

| 情况 | 策略 |
|-----|------|
| 数据量小，与预训练数据相似 | 只训练分类头 |
| 数据量小，与预训练数据不同 | 微调顶层 |
| 数据量大，与预训练数据相似 | 微调更多层 |
| 数据量大，与预训练数据不同 | 微调整个网络或从头训练 |

### 注意事项

- 微调时学习率应该比初始训练低10-100倍
- 始终先冻结基础模型，让分类头先适应
- 使用数据增强防止过拟合
- BatchNormalization层在迁移学习时保持`training=False`